# Machine Learning - Kubernetes

In this homework, we'll deploy the credit scoring model from the homework 5. We already have a docker image for this model - we'll use it for deploying the model to Kubernetes.

In [ ]:
import requests

In [ ]:
# download all the files at this folder_url and write them to the folder ./src

folder_url = 'https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2023/05-deployment/homework'
raw_url = 'https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/'

# get every file in the folder
response = requests.get(folder_url)

# convert the response text to JSON dictionary
folder_json = response.json()
print(folder_json.keys())

In [ ]:
# display the keys in the payload node
# print(folder_json['payload']['tree']['items'])

# for each item, download the file and write it to the folder ./src
for item in folder_json['payload']['tree']['items']:
    print(item['path'])
    file_name = item['path'].split('/')[-1]
    
    # get the github raw url for the folder url in order to download the file        
    file_url = raw_url + '/' + file_name
    print(file_url)
    file_response = requests.get(file_url)
    # write the file to the folder ./src with bytes
    with open('./src/' + file_name, 'wb') as f:
        f.write(file_response.content)
    

## Build the Docker image
- With a terminal, open the src folder. There should be a Dockerfile
  
```bash
docker build -t zoomcamp-model:hw10 .
```

## Question 1 - What is the credit score

- Run it to test that it's working locally:

```bash 
docker run -it --rm -p 9696:9696 zoomcamp-model:hw10
```
- And in another terminal, execute q6_test.py file:
```bash
python3 q6_test.py
```
You should see this:

```bash
{'get_credit': True, 'get_credit_probability': <value>}
```
Here <value> is the probability of getting a credit card. You need to choose the right one.

- 0.3269
- 0.5269
- 0.7269 *
- 0.9269


In [7]:
# run the test file to get the result
!python3 ./src/q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


## Stop the Docker container

In [9]:
# check the running containers
!docker ps

!docker stop ecd7a30f90c4   


CONTAINER ID   IMAGE                 COMMAND                  CREATED          STATUS          PORTS                    NAMES
ecd7a30f90c4   zoomcamp-model:hw10   "waitress-serve --li…"   39 seconds ago   Up 37 seconds   0.0.0.0:9696->9696/tcp   wizardly_bose
ecd7a30f90c4


## Install kubectl

```bash
# check the processor name amd or arm
uname -m

# Download the latest version of kubectl
sudo curl -Lo /usr/local/bin/kubectl https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl

# Make kubectl executable
sudo chmod +x /usr/local/bin/kubectl

# Verify installation
kubectl version --client
```


## Install kind

```bash
# Download kind
sudo curl -Lo /usr/local/bin/kind https://kind.sigs.k8s.io/dl/v0.20.0/kind-linux-amd64

# Make kind executable
sudo chmod +x /usr/local/bin/kind

# Verify installation
kind version
```


In [13]:
!kubectl version --output=json


{
  "clientVersion": {
    "major": "1",
    "minor": "25",
    "gitVersion": "v1.25.4",
    "gitCommit": "872a965c6c6526caa949f0c6ac028ef7aff3fb78",
    "gitTreeState": "clean",
    "buildDate": "2022-11-09T13:36:36Z",
    "goVersion": "go1.19.3",
    "compiler": "gc",
    "platform": "linux/amd64"
  },
  "kustomizeVersion": "v4.5.7"
}
The connection to the server localhost:8080 was refused - did you specify the right host or port?
kind v0.20.0 go1.20.4 linux/amd64


## Question 2 - What is the version of kind

In [14]:
!kind version

kind v0.20.0 go1.20.4 linux/amd64


## Create a cluster

- Now let's create a cluster with kind:

```bash
kind create cluster

```
- And check with kubectl that it was successfully created:

```bash
kubectl cluster-info
```

## Question 3 - What's CLUSTER-IP of the service that is already running there?

Use kubectl to get the list of running services.

In [18]:
!kubectl get services
# import requests

# response = requests.get('https://127.0.0.1:36891/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy')
# print(response.text)

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   84m


## Question 4 

To be able to use the docker image we previously created (zoomcamp-model:hw10), we need to register it with kind.

What's the command we need to run for that?

- kind create cluster
- kind build node-image
- kind load docker-image *
- kubectl apply

In [19]:
!kind load docker-image zoomcamp-model:hw10

Image: "zoomcamp-model:hw10" with ID "sha256:aa9f3a58297d139f06e678b54779c12db09f29c2733ed44ccfb5757cb4ab7a4c" not yet present on node "kind-control-plane", loading...


## Question 5 - 

Now let's create a deployment config (e.g. deployment.yaml)

```python
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit
spec:
  selector:
    matchLabels:
      app: credit
  replicas: 1
  template:
    metadata:
      labels:
        app: credit
    spec:
      containers:
      - name: credit
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace <Image>, <Memory>, <CPU>, <Port> with the correct values.

What is the value for <Port>? **9696**

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [20]:
 !kubectl apply -f ./src/deployment.yaml

deployment.apps/credit created


In [21]:
!kubectl get deployments

NAME     READY   UP-TO-DATE   AVAILABLE   AGE
credit   1/1     1            1           6m20s


## Question 6 -  What do we need to write instead of <???>?

Let's create a service for this deployment (service.yaml):

```python
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Use the deployment name from the `!kubectl get deployments` output  and apply the file as:

```bash
kubectl apply -f ./src/service.yaml
```

In [22]:
# apply the service.yaml file
!kubectl apply -f ./src/service.yaml

service/ozkary-credit-service created


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

kubectl port-forward service/<Service name> 9696:80
Run q6_test.py (from the homework 5) once again to verify that everything is working. You should get the same result as in Question 1.

In [24]:
!kubectl port-forward service/ozkary-credit-service 9696:80 &

Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
^C


In [25]:
!python3 ./src/q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


Autoscaling

Now we're going to use a HorizontalPodAutoscaler (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment credit --name credit-hpa --cpu-percent=20 --min=1 --max=3
```
You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:
```bash
NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-hpa   Deployment/credit   1%/20%    1         3         1          27s

```

TARGET column shows the average CPU consumption across all the Pods controlled by the corresponding deployment. Current CPU consumption is about 0% as there are no clients sending requests to the server.

In [26]:
!kubectl get hpa

NAME         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-hpa   Deployment/credit   <unknown>/20%   1         3         1          8m32s


## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing q6_test.py script by putting the operator that sends the request to the credit service into a loop.

```bash
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.**Use q6_test_loop.py**

## Question 7 - What was the maximum amount of the replicas during this test?

Run kubectl get hpa credit-hpa --watch command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test?

- 1
- 2
- 3
- 4

In [38]:
!kubectl get hpa credit-hpa

NAME         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-hpa   Deployment/credit   <unknown>/20%   1         3         1          19m


## Stop all the services

```bash
kubectl delete service,deployment ozkary-credit-service credit
```

In [39]:
!kubectl delete service,deployment ozkary-credit-service credit

service "ozkary-credit-service" deleted
deployment.apps "credit" deleted
Error from server (NotFound): services "credit" not found
Error from server (NotFound): deployments.apps "ozkary-credit-service" not found


In [40]:
# delete all the resources
!kubectl delete all --all

service "kubernetes" deleted
horizontalpodautoscaler.autoscaling "credit-hpa" deleted


In [41]:
# delete the cluster
!kind delete cluster

Deleting cluster "kind" ...
Deleted nodes: ["kind-control-plane"]


In [42]:
# kill the port forwarding process
!pkill -f 'kubectl port-forward'